In [1]:
import tensorflow as tf
tf.enable_eager_execution()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## Reading the dataset

In [5]:
raw_image_dataset = tf.data.TFRecordDataset('data/train.tfrecord')

# Create a dictionary describing the features.
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'angle': tf.io.FixedLenFeature([], tf.float32),
    'throttle': tf.io.FixedLenFeature([], tf.float32)
}

def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

In [54]:
def decode_jpeg(image_buffer, scope=None):
    """Decode a JPEG string into one 3-D float image Tensor.
    Args:
        image_buffer: scalar string Tensor.
        scope: Optional scope for name_scope.
    Returns:
        3-D float Tensor with values ranging from [0, 1).
    """
    with tf.name_scope(values=[image_buffer], name=scope,
                       default_name='decode_jpeg'):
        # Decode the string as an RGB JPEG.
        # Note that the resulting image contains an unknown height
        # and width that is set dynamically by decode_jpeg. In other
        # words, the height and width of image is unknown at compile-i
        # time.
        image = tf.image.decode_jpeg(image_buffer, channels=3)

        # After this point, all image pixels reside in [0,1)
        # until the very end, when they're rescaled to (-1, 1).
        # The various adjust_* ops all require this range for dtype
        # float.
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        return image

def _parse_fn(example_serialized, is_training=False):
    """Helper function for parse_fn_train() and parse_fn_valid()
    Each Example proto (TFRecord) contains the following fields:
    image/height: 462
    image/width: 581
    image/colorspace: 'RGB'
    image/channels: 3
    image/class/label: 615
    image/class/synset: 'n03623198'
    image/class/text: 'knee pad'
    image/format: 'JPEG'
    image/filename: 'ILSVRC2012_val_00041207.JPEG'
    image/encoded: <JPEG encoded string>
    Args:
        example_serialized: scalar Tensor tf.string containing a
        serialized Example protocol buffer.
    Returns:
        image_buffer: Tensor tf.string containing the contents of
        a JPEG file.
        label: Tensor tf.int32 containing the label.
        text: Tensor tf.string containing the human-readable label.
    """
    feature_map = {
        'image': tf.FixedLenFeature([], dtype=tf.string, default_value=''),
        'angle': tf.FixedLenFeature([], dtype=tf.float32, default_value=0.0),
        'throttle': tf.FixedLenFeature([], dtype=tf.float32, default_value=0.0),
    }
    
    parsed = tf.parse_single_example(example_serialized, feature_map)
    image = decode_jpeg(parsed['image'])
    image = tf.reshape(image, (1, 240, 360, 3))
    return (image, (parsed['angle'], parsed['throttle']))

In [55]:
def get_dataset(tfrecords_dir, subset, batch_size):
    """Read TFRecords files and turn them into a TFRecordDataset."""
    files = tf.matching_files(os.path.join(tfrecords_dir, '%s-*' % subset))
    shards = tf.data.Dataset.from_tensor_slices(files)
    shards = shards.shuffle(tf.cast(tf.shape(files)[0], tf.int64))
    shards = shards.repeat()
    dataset = shards.interleave(tf.data.TFRecordDataset, cycle_length=4)
    dataset = dataset.shuffle(buffer_size=8192)
    parser = partial(
        _parse_fn, is_training=True if subset == 'train' else False)
    dataset = dataset.apply(
        tf.data.experimental.map_and_batch(
            map_func=parser,
            batch_size=batch_size,
            num_parallel_calls=config.NUM_DATA_WORKERS))
    dataset = dataset.prefetch(batch_size)
    return dataset

In [56]:
parsed_image_dataset = raw_image_dataset.map(_parse_fn)

In [57]:
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Convolution2D, Convolution3D
from tensorflow.python.keras.layers import MaxPooling2D, MaxPooling3D
from tensorflow.python.keras.activations import relu
from tensorflow.python.keras.layers import Dropout, Flatten, Dense
from tensorflow.python.keras.layers import Cropping2D, Cropping3D
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping

In [58]:
weight_loss_angle = 0.9
weight_loss_throttle = 0.1

In [59]:
def create_2d_model(img_dims, crop_margin_from_top=80):
    tf.keras.backend.clear_session()

    img_in = Input(shape=(img_dims), name='img_in')

    x = img_in

    x = Cropping2D(((crop_margin_from_top, 0), (0, 0)))(x)

    # Define convolutional neural network to extract features from the images
    x = Convolution2D(filters=24, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=32, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)

    # Define decision layers to predict steering and throttle
    x = Flatten(name='flattened')(x)
    x = Dense(units=100, activation='linear')(x)
    x = Dropout(rate=.5)(x)
    x = Dense(units=50, activation='linear')(x)
    x = Dropout(rate=.5)(x)
    # categorical output of the angle
    angle_out = Dense(units=1, activation='linear', name='angle_out')(x)

    # continous output of throttle
    throttle_out = Dense(units=1, activation='linear', name='throttle_out')(x)

    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])

    model.summary()

    model.compile(optimizer='adam',
                loss={'angle_out': 'mean_squared_error',
                      'throttle_out': 'mean_squared_error'},
                loss_weights={'angle_out': weight_loss_angle,
                              'throttle_out': weight_loss_throttle},
                metrics=['mse', 'mae', 'mape'])

    return model

In [60]:
model = create_2d_model(img_dims=[240, 360, 3])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_in (InputLayer)             (None, 240, 360, 3)  0                                            
__________________________________________________________________________________________________
cropping2d (Cropping2D)         (None, 160, 360, 3)  0           img_in[0][0]                     
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 78, 178, 24)  1824        cropping2d[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 37, 87, 32)   19232       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (

In [61]:
model.fit(parsed_image_dataset, steps_per_epoch=128)

Instructions for updating:
Use tf.cast instead.
128/128 [==============================] - 9s 67ms/step - loss: 0.0375 - angle_out_loss: 0.0393 - throttle_out_loss: 0.0208 - angle_out_mean_squared_error: 0.0393 - angle_out_mean_absolute_error: 0.0762 - angle_out_mean_absolute_percentage_error: 42595612.0000 - throttle_out_mean_squared_error: 0.0208 - throttle_out_mean_absolute_error: 0.1052 - throttle_out_mean_absolute_percentage_error: 143917.3125
